### 使用GPT+ sliding windows算法 补全缺失值

In [2]:
# 计算message 和string中的token数量
from pyexpat.errors import messages
import tiktoken
# 2. Load an encoding 使用tiktoken.get_encoding() 按名称加载编码。第一次运行时，它将需要互联网连接进行下载。后续运行不需要互联网连接。
encoding = tiktoken.get_encoding("cl100k_base")
# 使用tiktoken.encoding_for_model()函数可以自动加载给定模型名称的正确编码。
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
# 3. Turn text into tokens with encoding.encode() The .encode() method converts a text string into a list of token integers.
encoding.encode("tiktoken is great!")
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """返回文本字符串中的Token数量"""
    num_tokens=0
    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
    num_tokens = len(encoding.encode(string))
    return num_tokens
def calculate_messages(messages:list) -> int:
    """返回总的message长度"""
    counter=0
    for mess in messages:
        counter+=num_tokens_from_string(mess.content,"cl100k_base")
    return counter
INCONTEXT_WINDOWS_GPT_3_5=16000
INCONTEXT_WINDOWS_MOONSHOT=30000
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.schema import HumanMessage, SystemMessage,AIMessage
from langchain_openai import ChatOpenAI
import os
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.schema import HumanMessage, SystemMessage,AIMessage
from langchain_openai import ChatOpenAI

# moonshot_chat.openai_api_key=
# moonshot_chat.openai_api_base="https://api.moonshot.cn/v1"
# moonshot_chat=ChatOpenAI(temperature=0,model="moonshot-v1-32k",openai_api_key='XXX',openai_api_base='https://api.moonshot.cn/v1')
# moonshot_chat.temperature
class MyException(Exception):
    def __init__(self,message):
        self.message=message
    def __str__(self):
        return self.message

class ChatGPTAnnotator:
    def __init__(self):
        self.chat = ChatOpenAI(temperature=0,
                               model="gpt-3.5-turbo",openai_api_key='XXX',
                               openai_api_base='https://gptnb.keqichen.top/v1')
        self.system_prompt="""You are an excellent AI psychotherapist. Please correct the *** part. Here is the explanation from the dictionary: The reasoning part in the conversation refers to the reasoning based on professional knowledge of psychological counseling, mainly explaining the therapist's motivation. Please modify the *** part in the patient's observation, which represents the Therapist's objective observation of the current patient's response. Please refer to the content of post_reasoning and pre_reasoning where annotating the *** part.Please replace "Chapter" and "book" with ‘methods’.There is an example for you."""
        self.fewshot0_0="""{'case_theme': 'The Friendly Stomach Discussion\n', 'background': 'The discussion revolves around questioning techniques and recognizing when one is going in the wrong direction.', 'pre_reasoning': 'The importance lies in the response or answer rather than the questions. If unsatisfied with the answer, try asking a different question. When stuck in a loop, changing the question can be beneficial.', 'topic_dialog': {'key': 'fewshot', 'method': 'Solution-Focused Brief Therapy(SFBT)', 'dialog': [{'id': '518-0', 'participant': 'Robert', 'text': ' In a week—well, nothing, but at the weekends …', 'observation': '***'}, {'id': '518-1', 'participant': 'Therapist', 'text': ' Uh-huh.', 'reasoning': '***'}, {'id': '518-2', 'participant': 'Robert', 'text': ' … it’s pretty much.', 'observation': 'Robert provides additional information'}, {'id': '518-3', 'participant': 'Therapist', 'text': ' Uh-huh. What do you do with your friends when you’re hanging around?', 'reasoning': '***'}], 'unit_id': 518}, 'post_reasoning': 'The questions help to create a detailed picture of exceptional times. Learning this technique requires practice and observation of experienced therapists. Mastery comes with time and experience, similar to learning to drive a car by actually driving.'}"""
        self.fewshot0_1="""{"dialog":[
    {
        "id": "518-0",
        "participant": "Robert",
        "text": " In a week—well, nothing, but at the weekends …",
        "observation": "Robert is hesitant to share details initially."
    },
    {
        "id": "518-1",
        "participant": "Therapist",
        "text": " Uh-huh.",
        "reasoning": "The Therapist acknowledges Robert's response, indicating a continuation of the conversation to explore further."
    },
    {
        "id": "518-2",
        "participant": "Robert",
        "text": " … it’s pretty much.",
        "observation": "Robert provides additional information"
    },
    {
        "id": "518-3",
        "participant": "Therapist",
        "text": " Uh-huh. What do you do with your friends when you’re hanging around?",
        "reasoning": "Therapist prompts Robert to share more about his social activities to understand Robert's social interactions and activities outside of his initial vague response"
    }
]}
"""
        self.fewshot1_0="""{'case_theme': 'Chapter Seven Working With Reluctant Clients and Challenging Situations', 'background': 'The chapter focuses on strategies to engage reluctant clients in counseling, especially those who are mandated or sent against their will. It discusses techniques to empathize with clients and turn challenging situations into achievable goals.', 'pre_reasoning': 'The chapter emphasizes the importance of converting reluctant clients into willing participants in counseling. It provides strategies for empathizing with clients who are sent to counseling against their will and offers tips for handling mandated clients, such as students sent for behavioral issues.', 'topic_dialog': {'key': 'fewshot', 'method': 'Solution-Focused Brief Therapy', 'dialog': [{'id': '183-21', 'participant': 'Mac', 'text': '[No response; continues to look at floor]', 'reference_labels': ['[No response; continues to look at floor]'], 'observation': '***'}, {'id': '183-22', 'participant': 'Therapist', 'text': ' Well, what do you think? By being here and meeting with me, passing would be your ultimate goal?', 'reasoning': '***'}, {'id': '183-23', 'participant': 'Mac', 'text': '[Nods head yes]', 'reference_labels': ['[Nods head yes]'], 'observation': '***'}, {'id': '183-24', 'participant': 'Therapist', 'text': ' Is that what you’d like to have happen? It doesn’t look real promising right now, but at least that would be something you would like to do?', 'reasoning': '***'}, {'id': '183-25', 'participant': 'Mac', 'text': '[Nods head yes]', 'reference_labels': ['[Nods head yes]'], 'observation': '***'}, {'id': '183-26', 'participant': 'Therapist', 'text': ' At least, if you can’t pass, we could at least go over the steps so it won’t happen again. Is that something you might think about or want to work on?', 'reasoning': '***'}, {'id': '183-27', 'participant': 'Mac', 'text': ' Yeah.', 'observation': '***'}, {'id': '183-28', 'participant': 'Therapist', 'text': ' So, am I hearing that you would like to develop some better habits? You didn’t say that, but that’s what I was getting from what you said.', 'reasoning': '***'}, {'id': '183-29', 'participant': 'Mac', 'text': ' Yeah, that would be good.', 'observation': '***'}], 'unit_id': 183}, 'post_reasoning': "The chapter highlights the importance of empathizing with reluctant clients and using techniques like 'They’re wrong, you’re right' to align with clients against those who referred them to counseling. It also discusses the need to establish positive goals with clients, even in challenging situations. Additionally, it acknowledges that not all clients may be receptive to the solution-focused brief therapy approach, and counselors may need to adapt their strategies accordingly."}"""
        self.fewshot1_1="""{
    "dialog": [
      {
        "id": "183-21",
        "participant": "Mac",
        "text": "[No response; continues to look at floor]",
        "reference_labels": ["[No response; continues to look at floor]"],
        "observation": "Mac's lack of verbal response and body language suggest a reluctance to engage, which is a common challenge when working with mandated clients."
      },
      {
        "id": "183-22",
        "participant": "Therapist",
        "text": " Well, what do you think? By being here and meeting with me, passing would be your ultimate goal?",
        "reasoning": "The therapist is attempting to connect with Mac by framing the counseling process in terms of a 'passing' goal, which may be more relatable and less threatening than discussing emotional or behavioral issues."
      },
      {
        "id": "183-23",
        "participant": "Mac",
        "text": "[Nods head yes]",
        "reference_labels": ["[Nods head yes]"],
        "observation": "Mac's non-verbal affirmative response is an encouraging sign that he may be more open to the process than his initial silence suggested."
      },
      {
        "id": "183-24",
        "participant": "Therapist",
        "text": " Is that what you’d like to have happen? It doesn’t look real promising right now, but at least that would be something you would like to do?",
        "reasoning": "The therapist is validating Mac's current feelings of hopelessness while also inviting him to consider a possible positive outcome, aligning with the SFBT's emphasis on empathizing with reluctant clients."
      },
      {
        "id": "183-25",
        "participant": "Mac",
        "text": "[Nods head yes]",
        "reference_labels": ["[Nods head yes]"],
        "observation": "Mac's continued non-verbal agreement to the therapist's questions suggests a growing willingness to participate in setting a positive goal for the counseling sessions."
      },
      {
        "id": "183-26",
        "participant": "Therapist",
        "text": " At least, if you can’t pass, we could at least go over the steps so it won’t happen again. Is that something you might think about or want to work on?",
        "reasoning": "The therapist is offering a pragmatic approach, focusing on learning from the situation to prevent future occurrences, which may be more appealing to Mac and help establish a positive and collaborative goal."
      },
      {
        "id": "183-27",
        "participant": "Mac",
        "text": " Yeah.",
        "observation": "Mac's verbal agreement marks a significant shift towards engagement, indicating that the therapist's empathetic and goal-oriented approach is resonating with him."
      },
      {
        "id": "183-28",
        "participant": "Therapist",
        "text": " So, am I hearing that you would like to develop some better habits? You didn’t say that, but that’s what I was getting from what you said.",
        "reasoning": "The therapist is summarizing and clarifying Mac's intentions, aiming to ensure they are on the same page and that Mac feels understood, which is crucial for building rapport and moving forward in the counseling process."
      },
      {
        "id": "183-29",
        "participant": "Mac",
        "text": " Yeah, that would be good.",
        "observation": "Mac's agreement to the therapist's summary confirms his readiness to work on developing better habits, demonstrating the effectiveness of establishing positive goals with clients."
      }
    ]
}
"""
        self.origin_message,self.token_count=self.get_init_message()

    def get_init_message(self):
        messages = [
        SystemMessage(
            content=self.system_prompt
        ),
        # HumanMessage(
        #     content="###There is an example:###"+fo_0
        # ),
        # AIMessage(
        #     content="###There is the result after annotation：###"+fp_0
        # ),
        HumanMessage(
            content="###There is an example: ###"+self.fewshot1_0
        ),
        AIMessage(
            content="###There is the result after annotation: ###"+self.fewshot1_1
        )]
        token_count=calculate_messages(messages)
        return messages,token_count
    def chat_with_gpt(self,input_json:str,max_try_times:int=3):
        messages,init_count=self.get_init_message()
        messages.append(HumanMessage(
            content="There is my json, please annotate it! You are the best! return a json with a 'dialog' key-value.: "+input_json 
        ))
        response=self.chat(messages)
        # response_metadata={'finish_reason': 'length', 'logprobs': None}
        final_response_content=response.content
        counter=0
        response_metadata={'finish_reason': 'stop', 'logprobs': None}
        while 'length' in response.response_metadata['finish_reason'] and counter<max_try_times:
            print(len(messages),response)
            messages.append(AIMessage(
                content=final_response_content
            ))
            messages.append(HumanMessage(
                content="Well done! Please go on: ###"
            ))
            response=self.chat(messages)
            final_response_content+=response.content
            counter+=1
            
        if counter==max_try_times:
            raise MyException("The response is too long! Max_try!!!!!!!")
        return final_response_content,response.response_metadata

In [3]:
import os
import json
def read_file(file_path:str):
    with open(file_path,"r",encoding="utf-8") as i_f:
        return json.load(i_f)
def write_file(file_path:str,obj,overwrite=False):
    if os.path.exists(file_path)==False or overwrite==True:
        with open(file_path,"w",encoding="utf-8") as o_f:
            json.dump(obj,o_f,ensure_ascii=False)
        print(f"写入{file_path}")
    else:
        print(f"{file_path}文件已经存在")
def check_file(file_path:str):
    return os.path.exists(file_path)

In [4]:
def build_id(number):
    return f"518-{number:03d}"

In [5]:
build_id(1)

'518-001'

In [6]:

def fill_missing_keys(input_file_json,output_file_json,file_name,save_path):
    # 对id进行校正，补全 不足的ID 比如 background post_reasoning pre_reasoning
    i_file_key_list=list(input_file_json.keys())
    o_file_key_list=list(output_file_json.keys())
    if "topic_dialog" not in i_file_key_list or set(i_file_key_list+["background"]) != set(o_file_key_list):
        print(f"input{file_name} {i_file_key_list} , output {file_name} {o_file_key_list}")
        key_list=list(set(i_file_key_list+["background"])-set(o_file_key_list))
        for key in key_list:
            if key in ['post_reasoning','pre_reasoning']:
                print(file_name,'reasoning_fix')
                output_file_json[key]=''.join(input_file_json[key])
            elif key in ['background']:
                print(file_name,'background')
                output_file_json[key]=''
            else:
                output_file_json[key]=input_file_json[key]
        if len(key_list)>0:
            print(f"补全{file_name}")
            write_file(save_path+"/"+file_name,output_file_json)


In [7]:
import copy
def fill_missing_dialog(input_file_json,output_file_json,file_name,save_path,windows_size=10):
    i_dialog=input_file_json["topic_dialog"]["dialog"]
    o_dialog=output_file_json["topic_dialog"]["dialog"]
    tmp_json=copy.deepcopy(output_file_json)
    o_dialog_id_list=[item["id"] for item in o_dialog]
    i_dialog_id_list=[item["id"] for item in i_dialog]
    if file_name.startswith("50_"):
        print(i_dialog,o_dialog,len(i_dialog),len(o_dialog))
    if set(o_dialog_id_list)!=set(i_dialog_id_list):
        print(file_name,len(i_dialog),len(o_dialog))
    if len(i_dialog)!=len(o_dialog):
        print(file_name,len(i_dialog),len(o_dialog),o_dialog)
    no_explanation_id_list=[]
    dialog_window=[]
    result_list=[]
    is_missing=False
    o_dialog=sorted(o_dialog,key=lambda x:int(x["id"].split("-")[-1]))
    for idx,ditem in enumerate(o_dialog):
        if "Therapist" in ditem["participant"] and "observation" in ditem.keys():
            # print(ditem.keys()) 注意python的if 条件判断中的or逻辑
            del ditem["observation"]## 构造错乱,请重新生成

        if "Therapist" not in ditem["participant"] and "reasoning" in ditem.keys():
            del ditem["reasoning"]# 构造错乱 请重新生成并做映射

        if 'reasoning' not in ditem.keys() and "observation" not in ditem.keys():
            if "Therapist" in ditem["participant"]:
                ditem["reasoning"] = "***"
            else:
                ditem["observation"]="***"
            no_explanation_id_list.append(ditem["id"])
            is_missing=True
        dialog_window.append(ditem)
        # 只有满足了窗口长度或者到了最后一个元素才处理
        if (idx % windows_size)==0 and idx!=0 or idx==len(o_dialog)-1:
            # assert len(processing_queue)==windows_size or idx==len(o_dialog)-1
            ## 进行处理逻辑
            if is_missing==True:## 补全
                ### chat逻辑
                tmp_json["topic_dialog"]["dialog"]=dialog_window
                # if ditem["id"].startswith("518") or ditem["id"].startswith("183"):
                #     print(tmp_json)
                result=main_chat(json.dumps(tmp_json,ensure_ascii=False,indent=4),file_name+"_"+str(idx))
                if result==None:
                    print(f"{file_name}无法处理")
                    break 
                else:
                    result_list.extend(list(result["dialog"]))
            else:
                ### 没有缺失
                result_list.extend(dialog_window)
            dialog_window=[]
            is_missing=False
    if result_list:
        tmp_json["topic_dialog"]["dialog"]=result_list
        try:
            assert len(result_list)==len(o_dialog)
        except:
            print("\n\n\n\n\n不相等!")
            print(len(result_list),len(o_dialog))
        if len(result_list)==len(o_dialog):
            write_file(save_path+"/"+file_name,tmp_json)
        else:
            print(f"{file_name}长度不相等")
    if no_explanation_id_list:
        print(no_explanation_id_list,len(result_list),len(o_dialog))

In [8]:
a=[1,2,3,4,5]
k=2
[a[i:i+k] for i in range(0,len(a),k)]

[[1, 2], [3, 4], [5]]

In [9]:
# def fill_missing_values(input_file_json,output_file_json,file_name,save_path):
#     # fill_key_path="/home/ckqsudo/code2024/0dataset/E-bed/English/z3_gpt_mapping.ipynb/修正key缓存输出文件夹"
#     # fill_missing_keys(input_file_json=input_file_json,output_file_json=output_file_json,file_name=file_name,save_path=fill_key_path)
    

In [10]:

cache_path=r"/home/ckqsudo/code2024/0dataset/E-bed/English/z3_gpt_mapping.ipynb/缓存文件夹"
output_cache_path=r"/home/ckqsudo/code2024/0dataset/E-bed/English/z3_gpt_mapping.ipynb/缓存输出文件夹"
save_path="/home/ckqsudo/code2024/0dataset/E-bed/English/z3_gpt_mapping.ipynb/修正dialog输出文件夹"
output_file_list=os.listdir(output_cache_path)
cache_file_list=os.listdir(cache_path)
def main(input_list=cache_file_list,output_list=output_file_list,file_list=cache_file_list,beg_idx=0):
    unhandle_file_list=list(set(input_list)-set(output_list))
    if len(unhandle_file_list)!=0:
        print(f"文件未处理{unhandle_file_list}")
    else:
        # 文件为0
        for file_name in sorted(file_list,key=lambda name:int(name.split("_")[0])):
            if int(file_name.split("_")[0])<beg_idx:
                continue
            if check_file(save_path+"/"+file_name):
                print(f"{file_name}文件已存在跳过")
                continue
            input_json=read_file(cache_path+"/"+file_name)
            output_json=read_file(output_cache_path+"/"+file_name)
            fill_missing_dialog(input_file_json=input_json,output_file_json=output_json,file_name=file_name,save_path=save_path,windows_size=5)

In [11]:
chatGPTAnnotator=ChatGPTAnnotator()
# test_result=chatGPTAnnotator.chat_with_gpt(chatGPTAnnotator.fewshot0_0)

In [12]:
from os import error
import time
chatGPTAnnotator=ChatGPTAnnotator()
def main_chat(json_str,messages):
    try:
        response,response_metadata=chatGPTAnnotator.chat_with_gpt(json_str)
        print(response)
        response_json=json.loads(response)
        
        if type(response_json)==list:
            response_json={"dialog":response_json}
        if "dialog" not in response_json.keys() and "topic_dialog" in response_json.keys():
            response_json=response_json["topic_dialog"]
        assert "dialog" in response_json.keys()
        time.sleep(1)
        return response_json
    except Exception as e: 
        print(messages,e,"出现错误，需要单独处理")
        time.sleep(10)
        return None
    

In [13]:
main(beg_idx=0)

0_cache.json文件已存在跳过
1_cache.json文件已存在跳过
2_cache.json文件已存在跳过
3_cache.json文件已存在跳过
4_cache.json文件已存在跳过
5_cache.json文件已存在跳过
6_cache.json文件已存在跳过
7_cache.json文件已存在跳过
8_cache.json文件已存在跳过
9_cache.json文件已存在跳过
10_cache.json文件已存在跳过
11_cache.json文件已存在跳过
12_cache.json文件已存在跳过
13_cache.json文件已存在跳过
14_cache.json文件已存在跳过
15_cache.json文件已存在跳过
16_cache.json文件已存在跳过
17_cache.json文件已存在跳过
18_cache.json文件已存在跳过
19_cache.json文件已存在跳过
20_cache.json文件已存在跳过
21_cache.json文件已存在跳过
22_cache.json文件已存在跳过
23_cache.json文件已存在跳过
24_cache.json文件已存在跳过
25_cache.json文件已存在跳过
26_cache.json文件已存在跳过
27_cache.json文件已存在跳过
28_cache.json文件已存在跳过
29_cache.json文件已存在跳过
30_cache.json文件已存在跳过
31_cache.json文件已存在跳过
32_cache.json文件已存在跳过
33_cache.json文件已存在跳过
34_cache.json文件已存在跳过
35_cache.json文件已存在跳过
36_cache.json文件已存在跳过
37_cache.json文件已存在跳过
38_cache.json文件已存在跳过
39_cache.json文件已存在跳过
40_cache.json文件已存在跳过
41_cache.json文件已存在跳过
42_cache.json文件已存在跳过
43_cache.json文件已存在跳过
44_cache.json文件已存在跳过
45_cache.json文件已存在跳过
46_cache.json文件已存在跳过
47_cache.json文件已存在跳过
48

In [14]:
# test_json=json.loads(test_result.content)

()

#### TO DO list
##### 纸巾
##### 